<pre>
1. Download the data from <a href='https://drive.google.com/file/d/15dCNcmKskcFVjs7R0ElQkR61Ex53uJpM/view?usp=sharing'>here</a>

2. Code the model to classify data like below image

<img src='https://i.imgur.com/33ptOFy.png'>

3. Write your own callback function, that has to print the micro F1 score and AUC score after each epoch.

4. Save your model at every epoch if your validation accuracy is improved from previous epoch. 

5. you have to decay learning based on below conditions 
        Cond1. If your validation accuracy at that epoch is less than previous epoch accuracy, you have to decrese the
               learning rate by 10%. 
        Cond2. For every 3rd epoch, decay your learning rate by 5%.
        
6. If you are getting any NaN values(either weigths or loss) while training, you have to terminate your training. 

7. You have to stop the training if your validation accuracy is not increased in last 2 epochs.

8. Use tensorboard for every model and analyse your gradients. (you need to upload the screenshots for each model for evaluation)

9. use cross entropy as loss function

10. Try the architecture params as given below. 
</pre>

<pre>
<b>Model-1</b>
<pre>
1. Use tanh as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use RandomUniform(0,1) as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>
<pre>
<b>Model-2</b>
<pre>
1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use RandomUniform(0,1) as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>
<pre>
<b>Model-3</b>
<pre>
1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use he_uniform() as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>
<pre>
<b>Model-4</b>
<pre>
1. Try with any values to get better accuracy/f1 score.  
</pre>
</pre>

In [2]:
import tensorflow as tf
tf.enable_eager_execution()
import numpy as np
import pandas as pd

In [5]:
from tensorflow.keras.layers import Dense,Input,Activation
from tensorflow.keras.models import Model
import random as rn

In [ ]:
C:\Users\nsuguru\Downloads\CallBack-master\CallBack-master

In [50]:
data = pd.read_csv("C:\\Users\\nsuguru\\Desktop\\data.csv")
from sklearn.model_selection import train_test_split

In [51]:
X=data.loc[:,'f1':'f2']
Y=data.loc[:,'label']

In [56]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, stratify=Y)

In [58]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(13400, 2)
(6600, 2)
(13400,)
(6600,)


In [59]:
class LossHistory(tf.keras.callbacks.Callback):
    
    def on_train_begin(self, logs={}):
        ## on begin of training, we are creating a instance varible called history
        ## it is a dict with keys [loss, acc, val_loss, val_acc]
        self.history={'loss': [],'acc': [],'val_loss': [],'val_acc': []}
        print('train_begin')
        
    def on_epoch_end(self, epoch, logs={}):
        print('\n','epoch_end')
        ## on end of each epoch, we will get logs and update the self.history dict
        self.history['loss'].append(logs.get('loss'))
        self.history['acc'].append(logs.get('acc'))
        if logs.get('val_loss', -1) != -1:
            self.history['val_loss'].append(logs.get('val_loss'))
        if logs.get('val_acc', -1) != -1:
            self.history['val_acc'].append(logs.get('val_acc'))

In [61]:
#Input layer
input_layer = Input(shape=(2,))
#Dense hidden layer
layer1 = Dense(50,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0,1,seed=20))(input_layer)
layer2 = Dense(50,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0,1,seed=30))(layer1)
layer3 = Dense(50,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0,1,seed=40))(layer2)
layer4 = Dense(50,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0,1,seed=50))(layer3)
layer5 = Dense(50,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0,1,seed=60))(layer4)


#output layer
output = Dense(10,activation='softmax',kernel_initializer=tf.keras.initializers.glorot_normal(seed=0))(layer5)
#Creating a model
model = Model(inputs=input_layer,outputs=output)


#Callbacks
history_own = LossHistory()

optimizer = tf.keras.optimizers.SGD(0.01)

model.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy'])

model.fit(X_train,Y_train,epochs=1, validation_data=(X_test,Y_test), batch_size=16, callbacks=[history_own])


ValueError: You are passing a target array of shape (13400, 1) while using as loss `categorical_crossentropy`. `categorical_crossentropy` expects targets to be binary matrices (1s and 0s) of shape (samples, classes). If your targets are integer classes, you can convert them to the expected format via:
```
from keras.utils import to_categorical
y_binary = to_categorical(y_int)
```

Alternatively, you can use the loss function `sparse_categorical_crossentropy` instead, which does expect integer targets.